In [1]:
# RUN THIS ONLY ONCE TO CREATE THE TABLES FOR THE DEMO (IMPRESSIONSTOORDERS)!!!!!
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket
from faker import Faker
from random import randrange

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()



In [2]:
from dotenv import load_dotenv
import json
from faker import Faker
from random import randrange

productIds = [707,708,711,712,714,715,716,717,718,722,738,739,742,743,747,748,779,780,781,782,783,784,792,793,794,795,796,797,798,799,800,801,808,809,810,813,822,835,836,838,858,859,860,864,865,867,868,869,870,873,874,875,876,877,880,881,883,884,885,886,889,891,892,893,894,895,896,899,900,904,905,907,908,909,910,913,916,917,918,920,924,925,926,935,936,937,938,939,940,944,945,947,948,949,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,996,997,998,999]

def generateClickEvent(impressionEvent):
    faker = Faker()
    click = {}
    click["impressionId"] = impressionEvent["impressionId"]
    click["clickDate"] = impressionEvent["impressionDate"]
    click["productId"] = impressionEvent["productId"]
    return click


def generateImpressionEvent(isUnsupportedBrowser, isAnomaly, productId):
    faker = Faker()
    impression = {}
    impression["impressionId"] = faker.uuid4()
    impression["impressionDate"] = faker.date_time_between(start_date="-1w", end_date="now").isoformat()
    if productId:
        impression["productId"] = str(productId)
    else:
        impression["productId"] = faker.random_element(productIds)

    if isUnsupportedBrowser:
        impression["browser"] = "unsupported"
    else:
        impression["browser"] = faker.random_element(["Edge", "Chrome", "Safari", "Firefox"])
    impression["browserVersion"] = faker.random_element(["10.2", "13.6", "8.6", "8.5", "11.2", "14.6", "6.6", "4.5"])
    impression["device"] = faker.random_element(["mobile", "computer", "tablet", "mobile", "computer"])
    impression["source"] = faker.random_element( ["organic", "bing", "google", "facebook"])
    impression["ip_address"] = faker.ipv4()
    impression["landing_page"] = faker.uri()
    if isAnomaly:
        impression["page_loading_seconds"] = faker.random_number(4)/100
    else:
        impression["page_loading_seconds"] = faker.random_number(4)/1000
    return impression


def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)


In [6]:
# generate fake impressions-orders events for demo
import pyodbc 
from dotenv import load_dotenv
import os

load_dotenv()

SQL_SERVER = os.getenv("SQL_SERVER") 
SQL_USER = os.getenv("SQL_USER")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DBNAME = os.getenv("SQL_DBNAME")
SQL_DRIVER = "SQL SERVER"
connectionString = f'DRIVER={SQL_DRIVER};SERVER={SQL_SERVER};DATABASE={SQL_DBNAME};UID={SQL_USER};PWD={SQL_PWD}'
conn = pyodbc.connect(connectionString)

import pandas as pd
sql_query = """
  SELECT  [SalesOrderID],[SalesOrderDetailID],[ProductID]
  FROM [SalesLT].[SalesOrderDetail]
"""

df = pd.read_sql_query(sql_query, conn)

C:\Users\dschlesinger\AppData\Local\Temp\ipykernel_38948\3837070683.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


In [ ]:
impressionsToOrdersDF = pd.DataFrame(columns=['impressionId', 'SalesOrderID', 'SalesOrderDetailID'])
for index, row in df.iterrows():
    SalesOrderID = row['SalesOrderID']
    SalesOrderDetailID = row['SalesOrderDetailID']
    productId = row['ProductID']
    
    #generate click and impression events
    impressionEvent = generateImpressionEvent(False, False, productId)
    sendToEventsHub(impressionEvent, producer_impressions)
    clickEvent = generateClickEvent(impressionEvent)
    sendToEventsHub(clickEvent, producer_clicks)

    #generate impressionToOrder event
    impressionToOrder = {}
    impressionToOrder["impressionId"] = impressionEvent["impressionId"]
    impressionToOrder["salesOrderId"] = SalesOrderID
    impressionToOrder["salesOrderDetailId"] = SalesOrderDetailID
    impressionsToOrdersDF.loc[len(impressionsToOrdersDF)] = impressionToOrder
impressionsToOrdersDF.to_csv("./data/impressionsToOrders.csv", index=False)
producer_impressions.close()
producer_clicks.close()

In [9]:
import pandas as pd
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
from azure.kusto.data.data_format import DataFormat
from azure.kusto.ingest import (
    BlobDescriptor,
    FileDescriptor,
    IngestionProperties,
    IngestionStatus,
    KustoStreamingIngestClient,
    ManagedStreamingIngestClient,
    QueuedIngestClient,
    StreamDescriptor,
)
load_dotenv()
AAD_TENANT_ID = os.getenv("AAD_TENANT_ID")
KUSTO_CLUSTER = os.getenv("KUSTO_CLUSTER")
KUSTO_INGEST_URI = os.getenv("KUSTO_INGEST_URI")
KUSTO_DATABASE = os.getenv("KUSTO_DATABASE")
KUSTO_MANAGED_IDENTITY_APP_ID = os.getenv("KUSTO_MANAGED_IDENTITY_APP_ID")
KUSTO_MANAGED_IDENTITY_SECRET = os.getenv("KUSTO_MANAGED_IDENTITY_SECRET")

# Connect to adx using AAD app registration for ingestion - use ingest URI
kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication(KUSTO_INGEST_URI, KUSTO_MANAGED_IDENTITY_APP_ID, KUSTO_MANAGED_IDENTITY_SECRET,  AAD_TENANT_ID)
client = KustoClient(kcsb)
kusto_db = KUSTO_DATABASE

DESTINATION_TABLE = "impressionsToOrders"
DESTINATION_TABLE_COLUMN_MAPPING = "impressionsToOrders_mapping"

client = QueuedIngestClient(kcsb)

# there are a lot of useful properties, make sure to go over docs and check them out
ingestion_props = IngestionProperties(
    database=KUSTO_DATABASE,
    table=DESTINATION_TABLE,
    data_format=DataFormat.CSV,
    # in case status update for success are also required (remember to import ReportLevel from azure.kusto.ingest)
    # report_level=ReportLevel.FailuresAndSuccesses,
    # in case a mapping is required (remember to import IngestionMappingKind from azure.kusto.data.data_format)
    # ingestion_mapping_reference="{json_mapping_that_already_exists_on_table}",
    # ingestion_mapping_kind= IngestionMappingKind.JSON,
)

df = pd.read_csv("./data/impressionsToOrders.csv")

client.ingest_from_dataframe(df, ingestion_properties=ingestion_props)

IngestionResult(status=IngestionStatus.QUEUED, database=adx-demo-rta, table=impressionsToOrders2, source_id=4d3735b9-74d2-4d7b-b6b6-cc68caf2bd37, blob_uri=https://7ufldadxdlhousedemoadx01.blob.core.windows.net/20240101-ingestdata-e5c334ee145d4b4-0/adx-demo-rta__impressionsToOrders2__4d3735b9-74d2-4d7b-b6b6-cc68caf2bd37__df_2795620084336_1704107554_353ccd23-6d63-40d3-94a3-4727c056309a.csv.gz)